## Machine Learning Accelerator Computer Vision - Final Project - Day 3

In this project, you will train Convolutional Neural Networks to correctly classify images of luggages. The competition is hosted here: https://mlu.corp.amazon.com/contests/redirect/50

We have a total of 5 classes and they are:
* Class 0: Inconclusive
* Class 1: Two wheels
* Class 2: Four wheels
* Class 3: Not a luggage
* Class 4: Zero wheels

__You can try to improve your previous model or try VGGNet and ResNet. MLA-CV-DAY2-AlexNet.ipynb and MLA-CV-DAY3-ResNet.ipynb will be helpful__

Let's import all the libraries. Before that, make sure you have installed the required version our libraries as below.

In [ ]:
! pip install -q torch==1.8.0
! pip install -q torchvision==0.9.0
! pip install -q d2l==0.16.01
! pip install -q numpy==1.19.5

In [ ]:
import pandas as pd
from glob import glob
import os
from shutil import copyfile
from torch.utils.data import Dataset
from PIL import Image
import numpy as np
from numpy.random import permutation
import matplotlib.pyplot as plt

from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18,resnet34,densenet121
from torchvision.models.inception import inception_v3
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import pickle
%matplotlib inline
import multiprocessing
cpu_count = multiprocessing.cpu_count()
is_cuda = torch.cuda.is_available()
is_cuda

### 1. Reading the dataset and creating loaders:

Your final project dataset is now under the __"data/final_project_dataset"__ folder. Over there you will see the __training__, __validation__ and __test__ folders. Let's start creating the data transforms and loaders below. In this project, images come in different sizes and we will resize them to 224x224 and normalize pixel values.

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

path = '../../data/final_project_dataset'
train_path = os.path.join(path, 'train')
val_path = os.path.join(path, 'validate')
test_path = os.path.join(path, 'test')

In [ ]:
def count_images_in_folder(path):
    import os
    ans = {}
    for clas in os.listdir(path):
        ans[clas] = len(os.listdir(os.path.join(path, clas)))
    return ans 

In [ ]:
train_dist = count_images_in_folder(train_path)
print("Training distribution:", train_dist)
val_dist = count_images_in_folder(val_path)
print("Validation distribution:", val_dist)

In [ ]:
simple_tranform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

simple_tranform_incp = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.RandomResizedCrop(size=(224, 224)),
    transforms.ColorJitter(brightness=.5,contrast=0.3, saturation=0.2, hue=.3),
    transforms.RandomRotation(degrees=(0, 30)),
    transforms.RandomAdjustSharpness(sharpness_factor=2),
    transforms.RandomAutocontrast(),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

In [ ]:
batch_size = 512
train_loader = DataLoader(
    ImageFolder(train_path, transform=simple_tranform),
    batch_size=batch_size, shuffle=False )

val_loader = DataLoader(
    ImageFolder(val_path, transform=simple_tranform),
    batch_size=batch_size, shuffle=False)

test_loader = DataLoader(
    ImageFolder(test_path, transform=simple_tranform),
    batch_size=batch_size, shuffle=False)

train_loader_incp = DataLoader(
    ImageFolder(train_path, transform=simple_tranform_incp),
    batch_size=batch_size, shuffle=False )

val_loader_incp = DataLoader(
    ImageFolder(val_path, transform=simple_tranform_incp),
    batch_size=batch_size, shuffle=False)

test_loader_incp = DataLoader(
    ImageFolder(test_path, transform=simple_tranform_incp),
    batch_size=batch_size, shuffle=False)

### 2. Training and Validation

In this part, you will use a simple Convolutional Neural Network. You can start with a network that we discuss in the class today. You will use the __train_loader__ and __validation_loader__ from above. 

Some important notes:
* There are 5 classes in this project, so adjust the network's last dense layer for that
* Once the tranining runs without errors, experiment with batch_size (in the data loaders part) and learning rate in your code. 

In [ ]:
class FeaturesDataset(Dataset):
    
    def __init__(self,featlst1,featlst2,featlst3,labellst):
        self.featlst1 = featlst1
        self.featlst2 = featlst2
        self.featlst3 = featlst3
        self.labellst = labellst
        
    def __getitem__(self,index):
        return (self.featlst1[index],self.featlst2[index],self.featlst3[index],self.labellst[index])
    
    def __len__(self):
        return len(self.labellst)
    
def fit(epoch,model,data_loader,phase='training',volatile=False):
    if phase == 'training':
        model.train()
    if phase == 'validation':
        model.eval()
        volatile=True
    running_loss = 0.0
    running_correct = 0
    for batch_idx , (data1,data2,data3,target) in enumerate(data_loader):
        if is_cuda:
            data1,data2,data3,target = data1.cuda(),data2.cuda(),data3.cuda(),target.cuda()
        data1,data2,data3,target = Variable(data1,volatile),Variable(data2,volatile),Variable(data3,volatile),Variable(target)
        if phase == 'training':
            optimizer.zero_grad()
        output = model(data1,data2,data3)
        w = torch.tensor([0.9495, 0.5931, 0.6861, 0.9784, 0.7929]).cuda()
        loss = F.cross_entropy(output,target,w)
        
        
        
        running_loss += F.cross_entropy(output,target,size_average=False).data.item()
        preds = output.data.max(dim=1,keepdim=True)[1]
        running_correct += preds.eq(target.data.view_as(preds)).cpu().sum()
        if phase == 'training':
            loss.backward()
            optimizer.step()
    
    loss = running_loss/len(data_loader.dataset)
    accuracy = 100. * running_correct/len(data_loader.dataset)
    
    print(f'{phase} loss is {loss:{5}.{2}} and {phase} accuracy is {running_correct}/{len(data_loader.dataset)}{accuracy:{10}.{4}}')
    return loss,accuracy

class LayerActivations():
    features=[]
    
    def __init__(self,model):
        self.features = []
        self.hook = model.register_forward_hook(self.hook_fn)
    
    def hook_fn(self,module,input,output):
        #out = F.avg_pool2d(output, kernel_size=8)
        self.features.extend(output.view(output.size(0),-1).cpu().data)

    
    def remove(self):
        
        self.hook.remove()
    

In [ ]:
#Create ResNet model
my_resnet = resnet34(pretrained=True)

if is_cuda:
    my_resnet = my_resnet.cuda()

my_resnet = nn.Sequential(*list(my_resnet.children())[:-1])

for p in my_resnet.parameters():
    p.requires_grad = False

#Create inception model

my_inception = inception_v3(pretrained=True)
my_inception.aux_logits = False
if is_cuda:
    my_inception = my_inception.cuda()
for p in my_inception.parameters():
    p.requires_grad = False

#Create densenet model

my_densenet = densenet121(pretrained=True).features
if is_cuda:
    my_densenet = my_densenet.cuda()
    
for p in my_densenet.parameters():
    p.requires_grad = False

In [ ]:
### For ResNet

trn_labels = []
trn_resnet_features = []
for d,la in train_loader:
    o = my_resnet(Variable(d.cuda()))
    o = o.view(o.size(0),-1)
    trn_labels.extend(la)
    trn_resnet_features.extend(o.cpu().data)
val_labels = []
val_resnet_features = []
for d,la in val_loader:
    o = my_resnet(Variable(d.cuda()))
    o = o.view(o.size(0),-1)
    val_labels.extend(la)
    val_resnet_features.extend(o.cpu().data)

# ### For Inception

trn_inception_features = LayerActivations(my_inception.Mixed_7c)
for da,la in train_loader_incp:
    _ = my_inception(Variable(da.cuda()))

trn_inception_features.remove()

val_inception_features = LayerActivations(my_inception.Mixed_7c)
for da,la in val_loader_incp:
    _ = my_inception(Variable(da.cuda()))

val_inception_features.remove()

### For Densenet


trn_densenet_features = []
for d,la in train_loader:
    o = my_densenet(Variable(d.cuda()))
    o = o.view(o.size(0),-1)
    
    trn_densenet_features.extend(o.cpu().data)
    

val_densenet_features = []
for d,la in val_loader:
    o = my_densenet(Variable(d.cuda()))
    o = o.view(o.size(0),-1)
    val_densenet_features.extend(o.cpu().data)

In [ ]:
trn_feat_dset = FeaturesDataset(trn_resnet_features,trn_inception_features.features,trn_densenet_features,trn_labels)
val_feat_dset = FeaturesDataset(val_resnet_features,val_inception_features.features,val_densenet_features,val_labels)
trn_feat_loader = DataLoader(trn_feat_dset,batch_size=512,shuffle=True)
val_feat_loader = DataLoader(val_feat_dset,batch_size=512)

In [ ]:
class EnsembleModel(nn.Module):
    
    def __init__(self,out_size,training=True):
        super().__init__()
        self.fc1 = nn.Linear(512,512)
        self.fc2 = nn.Linear(131072,512)
        self.fc3 = nn.Linear(50176,512)
        self.fc4 = nn.Linear(512,out_size)

    def forward(self,inp1,inp2,inp3):
        out1 = self.fc1(F.dropout(inp1,training=self.training))
        out2 = self.fc2(F.dropout(inp2,training=self.training))
        out3 = self.fc3(F.dropout(inp3,training=self.training))
        out = out1 + out2 + out3
        out = self.fc4(F.dropout(out,training=self.training, p=0.75))
        return out

In [ ]:
torch.seed()

In [ ]:
em = EnsembleModel(5)
if is_cuda:
    em = em.cuda()
optimizer = optim.Adam(em.parameters(),lr=0.01)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10)


In [ ]:
train_losses , train_accuracy = [],[]
val_losses , val_accuracy = [],[]
for epoch in range(1,20):
    epoch_loss, epoch_accuracy = fit(epoch,em,trn_feat_loader,phase='training')
    val_epoch_loss , val_epoch_accuracy = fit(epoch,em,val_feat_loader,phase='validation')
    train_losses.append(epoch_loss)
    train_accuracy.append(epoch_accuracy)
    val_losses.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)
    scheduler.step()

### 3. Reading the test data and getting predictions

After you trained your model, it is time to read our test set and make predictions with it. Then, we will write our predictions to a csv file that thas "ID" and "label" columns. This will be our final project submission file.

In [ ]:
import pandas as pd

test_df = pd.read_csv("../../data/final_project_dataset/public_test_features.csv")
print(test_df.head())

In [ ]:
import numpy as np

file_index = 0
files = ImageFolder(test_path, transform=simple_tranform).samples
test_df["label"] = np.nan
# test_df["conf"] = np.nan

In [ ]:
### For ResNet

test_resnet_features = []
for d,la in test_loader:
    o = my_resnet(Variable(d.cuda()))
    o = o.view(o.size(0),-1)
    test_resnet_features.extend(o.cpu().data)

# ### For Inception


test_inception_features = LayerActivations(my_inception.Mixed_7c)
for da,la in test_loader_incp:
    _ = my_inception(Variable(da.cuda()))

test_inception_features.remove()

### For Densenet

test_densenet_features = []
for d,la in test_loader:
    o = my_densenet(Variable(d.cuda()))
    o = o.view(o.size(0),-1)
    test_densenet_features.extend(o.cpu().data)

In [ ]:
class FeaturesDatasetTest(Dataset):
    
    def __init__(self,featlst1,featlst2,featlst3):
        self.featlst1 = featlst1
        self.featlst2 = featlst2
        self.featlst3 = featlst3
        
        
    def __getitem__(self,index):
        return (self.featlst1[index],self.featlst2[index],self.featlst3[index])
    
    def __len__(self):
        return len(self.featlst1)



test_feat_dset = FeaturesDatasetTest(test_resnet_features,test_inception_features.features,test_densenet_features)

test_feat_loader = DataLoader(test_feat_dset,batch_size=512, shuffle=False)

In [ ]:
with torch.no_grad():
   
    em.eval()
    volatile=True

    for batch_idx , (data1,data2,data3) in enumerate(test_feat_loader):
        if is_cuda:
            data1,data2,data3 = data1.cuda(),data2.cuda(),data3.cuda()
        data1,data2,data3 = Variable(data1,volatile),Variable(data2,volatile),Variable(data3,volatile)

        output = em(data1,data2,data3)
        preds = output.data.max(dim=1,keepdim=True)[1]
        confs = output.data.max(dim=1,keepdim=True)[0]
        
        for pred, conf in zip(preds, confs):
            test_df.loc[test_df["ASIN"]==files[file_index][0].split("/")[-1].split(".jpg")[0], "label"] = int(pred.data.item())
#             test_df.loc[test_df["ASIN"]==files[file_index][0].split("/")[-1].split(".jpg")[0], "conf"] = conf.data.item()
            file_index += 1

Let's load the test images and start creating our csv file. Here, we add a new column named "label". We write model's predictions to this column. 

## 4. Submit your predictions

This is the last step. The code block below will create a csv file: __final_project.csv__ with ID and label (your predictions) columns and then you will submit it to the leaderboard: https://mlu.corp.amazon.com/contests/redirect/50 "ASIN" column is not needed, so we drop it.

In [ ]:
import pandas as pd

test_df.drop(columns=["ASIN"], inplace=True)

test_df.to_csv("final_project.csv", index=False)

In [ ]:
test_df.to_csv("final_project.csv", index=False)

In [ ]:
test_df